In [96]:
"""
NOTES:

- the Cholesky decomposition over finite fields does not exist for d >= 3 (it does exist for d = 2) for d x d matrices
- however, we can find a general (non lower triangular) decomposition U = AA^* using BaseChangeToCanonical in the GAP `forms` package.
- writing \tilde{\rho}(g) = A^*\rho(g)A^*.inverse() gives unitary representations over finite fields

""";

In [77]:
U_0 = matrix(GF(3^2),[[1,2],[2,1]]); U_0

[1 2]
[2 1]

In [78]:
U_1 = matrix(GF(3^2),[[1,2,2],[2,1,2],[2,2,1]]); U_1

[1 2 2]
[2 1 2]
[2 2 1]

In [79]:
U_2 = matrix(GF(3^2),[[0,1,2],[1,0,1],[2,1,0]]); U_2

[0 1 2]
[1 0 1]
[2 1 0]

In [74]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(A.base_ring(),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [75]:
#define finite field F_9 we are working over
q = 3
F = GF(q^2)
z = F.gen()
w = F.subfield(1).multiplicative_generator()

In [69]:
import re
def convert_to_sage_gens(expression, q):
    
    # Convert Z(q) to w and Z(q^2) to z
    expression = re.sub(r'Z\((\d+)\)', str('w'), expression)  # Z(q)
    expression = re.sub(r'Z\((\d+)\^2\)', str('z'), expression)  # Z(q^2)
    expression = re.sub(r'\^', '**', expression) # Convert all occurrences of ^ to **
    
    return expression

In [93]:
#just use gap.eval to do gap evaluations in sequence
def base_change_matrix(U,q):
    gap.eval('LoadPackage("forms");')
    U_str = str([list(row) for row in U])
    gap.eval(f"u := Z({q})^0*{U_str};")
    gap.eval(f"form := HermitianFormByMatrix(u, GF({q}^2));")
    b_gap = gap.eval("b := BaseChangeToCanonical(form);")
    converted_expr = convert_to_sage_gens(b_gap, q)
    b = matrix(F,eval(converted_expr))
    return b

In [104]:
#verify base change for U_1
b1 = base_change_matrix(U_1,q)
A = b1.inverse()
A_star = conjugate_pos_char(A).transpose()
U_1 == A*A_star

True

In [105]:
#verify base change for U_2
b2 = base_change_matrix(U_2,q)
A = b2.inverse()
A_star = conjugate_pos_char(A).transpose()
U_2 == A*A_star

True